In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget


from pathlib import Path

import fiona
import numpy as np
from sqlalchemy import create_engine

from ppcollapse.utils.config import ConfigManager
from ppcollapse.utils.database import get_image

RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

# Load configuration
config = ConfigManager()
db_engine = create_engine(config.db_url)

shape_dir = Path("data/SHAPEFILES_adj")
file_ext = ".shp"

image_id = 32219

year = 2023
year_dir = shape_dir / f"{year}_qgis"

files = list(year_dir.glob(f"*{file_ext}"))

In [ ]:
img = get_image(image_id=image_id, config=config)


In [ ]:
file = files[0]

In [ ]:
with fiona.open(file) as src:
    polygons = [feature["geometry"] for feature in src]

In [ ]:
poly = polygons[0]